## 误差反向传播法

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    
apple = 100
apple_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()
# Forward pass
apple_price = mul_apple_layer.forward(apple, apple_num)  # 100 * 3
total_price = mul_tax_layer.forward(apple_price, tax)  # 300 * 1.1
print(total_price)  # 330.0
# Backward pass
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)  # dtotal_price
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # dapple_price
print(dapple, dapple_num, dtax)  # 3.3 110.0 300.0

330.0
3.3000000000000003 110.00000000000001 300


In [3]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
# Example usage of AddLayer
add_layer = AddLayer()
# Forward pass
x = 2.0
y = 3.0

result = add_layer.forward(x, y)  # 2.0 + 3.0
print(result)  # 5.0
# Backward pass
dout = 1
dx, dy = add_layer.backward(dout)  # dresult
print(dx, dy)  # 1.0 1.0

5.0
1 1


In [4]:
import numpy as np

class ReLU:
    def __init__(self):
        self.mask = None
        pass
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dx = dout.copy()
        dx[self.mask] = 0
        return dx
    
# Example usage of ReLU
relu_layer = ReLU()
# Forward pass
x = np.array([-1.0, 2.0, -3.0, 4.0])
out = relu_layer.forward(x)  # Apply ReLU
print(out)  # [0. 2. 0. 4.]
# Backward pass
dout = np.array([1.0, 1.0, 1.0, 1.0])  # Gradient from next layer
dx = relu_layer.backward(dout)  # Backpropagate through ReLU
print(dx)  # [0. 1. 0. 1.]
    
    

[0. 2. 0. 4.]
[0. 1. 0. 1.]


In [5]:
class SigmoidLayer:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out
    
    def backward(self, dout):
        dx = dout * (self.out * (1 - self.out))
        return dx
# Example usage of SigmoidLayer

sigmoid_layer = SigmoidLayer()
sigmoid_layer.forward(np.array([0.0, 1.0, -1.0]))  # Forward pass
# Backward pass
dout = np.array([1.0, 1.0, 1.0])  # Gradient from next layer
dx = sigmoid_layer.backward(dout)  # Back propagate through Sigmoid
print(dx)  # Gradient after Sigmoid

[0.25       0.19661193 0.19661193]


In [7]:
X = np.random.rand(2)
W  = np.random.rand(2, 3)
B = np.random.rand(3)

X.shape
W.shape
B.shape

Y = np.dot(X, W) + B

Y

array([0.96804405, 0.72819453, 1.06791734])

In [15]:
from common.functions import softmax, cross_entropy_error

class AffineLayer:
    def __init__(self, W, B):
        self.W = W
        self.B = B
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.B
        return out
    
    def backward(self, dout):
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = np.dot(dout, self.W.T)
        return dx
    
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [14]:
import sys, os
sys.path.append(os.pardir)  # Adds the parent directory to the system path
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # Initialize weights and biases
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # Create layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = AffineLayer(self.params['W1'], self.params['b1'])
        self.layers['ReLU'] = ReLU()
        self.layers['Affine2'] = AffineLayer(self.params['W2'], self.params['b2'])
        
        # Loss layer
        self.loss_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.loss_layer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    def gradient(self, x, t):
        # Forward pass
        self.loss(x, t)
        
        # Backward pass
        dout = 1
        dout = self.loss_layer.backward(dout)
        
        for layer in reversed(self.layers.values()):
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [16]:
import sys, os
sys.path.append(os.pardir)  # Adds the parent directory to the system path

from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical= network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + " : " + str(diff))  # Should be close to 0

W1 : 2.0872346495371502e-07
b1 : 2.292055180810464e-06
W2 : 5.504857720793838e-09
b2 : 1.3973923611160232e-07


In [17]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # Compute gradient
    grad = network.gradient(x_batch, t_batch)
    
    # Update parameters
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # Record loss
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # Print progress
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"Iteration {i}, Loss: {loss}, Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")

Iteration 0, Loss: 2.3021022751426874, Train Accuracy: 0.07266666666666667, Test Accuracy: 0.0722
Iteration 600, Loss: 0.24694378554793744, Train Accuracy: 0.9043, Test Accuracy: 0.9063
Iteration 1200, Loss: 0.30523167019327335, Train Accuracy: 0.92445, Test Accuracy: 0.9262
Iteration 1800, Loss: 0.19205024498963266, Train Accuracy: 0.9360166666666667, Test Accuracy: 0.9358
Iteration 2400, Loss: 0.15361915994946782, Train Accuracy: 0.9445, Test Accuracy: 0.9442
Iteration 3000, Loss: 0.1194670700324223, Train Accuracy: 0.9518666666666666, Test Accuracy: 0.9494
Iteration 3600, Loss: 0.07608735052061459, Train Accuracy: 0.9559333333333333, Test Accuracy: 0.9544
Iteration 4200, Loss: 0.18232933638618884, Train Accuracy: 0.9616833333333333, Test Accuracy: 0.9562
Iteration 4800, Loss: 0.05159220218785432, Train Accuracy: 0.9653833333333334, Test Accuracy: 0.9592
Iteration 5400, Loss: 0.060470133134077016, Train Accuracy: 0.9685833333333334, Test Accuracy: 0.9616
Iteration 6000, Loss: 0.19225